In [1]:
# Brian-setup
from brian2 import BrianLogger, start_scope, NeuronGroup, Synapses, network_operation, run, StateMonitor, SpikeMonitor, stop, rand
from brian2.units import ms, second
from brian2 import figure, grid, axhline, xlabel, ylabel, plot, legend, title, subplot,zeros, ones, arange, xticks, xlim, ylim

from brian2 import prefs, BrianLogger
prefs.logging.file_log_level = 'WARNING'
BrianLogger.file_handler.setLevel(prefs.logging.file_log_level)

# Scientific libs
import numpy as np
from scipy.signal import welch
from scipy import fftpack
import matplotlib.pyplot as plt
# make jupyter work with plots in cells
%matplotlib inline
plt.rcParams.update({'font.size': 16})

# Misc. libs
import random
import time

In [ ]:
# may be used to assess whether the population is firing synchronously.
def fft_determine_main_frequencies(t, x, sample_duration, threshold=70, do_plot=False, consider_convergence=True):
    X = fftpack.fft(x)
    freqs = fftpack.fftfreq(len(x)) * len(x)/float(sample_duration)
    
    main_freqs = []
    freq_highest_magnitude = 0
    highest_magnitude = 0
    for i, val in enumerate(np.abs(X[1:])):
        if(val >= threshold):
            main_freqs += [freqs[i+1]]  # +1 because the first value is omitted in X
            if(val > highest_magnitude):
                freq_highest_magnitude = main_freqs[-1]
    freq_highest_magnitude = np.abs(freq_highest_magnitude)
    main_freqs = np.abs(main_freqs)

    print('significant firing rates of magnitude >= {}:'.format(threshold), np.abs(main_freqs))
    
    freq_lim_for_plot = 110  # hardcoded value for when no frequencies are above the magnitude threshold
    if(len(main_freqs) > 0):
        freq_lim_for_plot = np.max(np.abs(main_freqs))+1
    
    if(do_plot and (len(main_freqs) > 0 or not consider_convergence)):
        fig, ax = plt.subplots()
        ax.plot(t, x)
        ax.set_xlabel('Time [s]')
        ax.set_ylabel('Signal amplitude');
        
        fig, ax = plt.subplots()
        ax.plot(X[1:])
        ax.set_xlabel('Frequency Space')
        ax.set_ylabel('Frequency Domain (Spectrum) Magnitude');
        
        fig, ax = plt.subplots()
        ax.stem(freqs[1:], np.abs(X[1:]))
        ax.set_xlabel('Frequency in Hertz [Hz]')
        ax.set_ylabel('Frequency Domain (Spectrum) Magnitude')
        ax.set_xlim(1, freq_lim_for_plot)
        ax.set_ylim(-5, 110)
        
    return {0: freq_highest_magnitude, 1: main_freqs}

In [2]:
def determine_main_frequency(t, x, do_plot=False, nperseg=4096):
    fs = 1000.0  # samples / second
    f, Pxx_den = welch(x, fs, nperseg=nperseg)
    mean_Pxx_den = np.mean(Pxx_den, axis=0)
    index_max_en = list(mean_Pxx_den).index(np.max(mean_Pxx_den))
    f_max_en = f[index_max_en]
    
    if(do_plot):
        fig = figure(figsize=(8, 4))
        fig.subplots_adjust(top=0.99)
        plt.semilogy(f, normalise(mean_Pxx_den))
        plt.ylim([0.5e-2, 1])
        plt.xlim(0, 50)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Avg. PSD ($v^2$/Hz)')
        plt.grid(True)
        plt.tight_layout();
        fname = './figures/' + str(time.time()).replace('.', '_') + '_PSD_f_max_' \
                + '{:4.2f}'.format(f_max_en).replace('.', '_') + '.eps'
        fig.savefig(fname, format='eps', dpi=1200)
#         plt.title('Energy by frequency ($x_{f_{max}}$'+r'$\approx$'+'${:4.2f}$'.format(f_max_en)+'$\ Hz$)')
    
    return f_max_en

def normalise(x):
    return x/np.max(np.abs(x))

In [3]:
def create_and_run_network_model(a, b, ws_e, ws_i, n, tau_g, steps, debug=False):
    start_scope()

    tau = 1*ms;
    c = -65;
    d_e=8; d_i=2;
    a_i=0.1; b_i=0.25;

    eqs_exc = '''
    dv/dt = (0.04*v**2 + 5*v + 140 - u +  I_tot_e + I_ext)/ tau : 1
    du/dt = a * (b*v - u)/tau : 1

    I_tot_e = I_tot_e_e + I_tot_i_e : 1

    I_tot_e_e : 1
    I_tot_i_e : 1
    I_ext : 1

    oper_ctr : 1
    '''

    eqs_inh = '''
    dv/dt = (0.04*v**2 + 5*v + 140 - u + I_tot_i + I_ext) / tau : 1
    du/dt = a_i * (b_i*v - u)/tau : 1

    I_tot_i = I_tot_e_i + I_tot_i_i : 1

    I_tot_e_i : 1
    I_tot_i_i : 1
    I_ext : 1
    '''

    n_excit = int(0.8 * n)
    n_inhib = n-n_excit
    if(debug):
        print('n_excit', n_excit)
        print('n_inhib', n_inhib)

    # using the euler-method in the numerical integration has been tested for this particular setup,
    #  and verified to only have a very minor (< 2 %) effect on signal delay propagation as when 
    #  compared to using RK4 when propagating a signal through 100 nodes in an Izhikevich-network
    G_excit = NeuronGroup(n_excit, eqs_exc, threshold='v>30', 
                    reset='''v=c; u=u+d_e;''', 
                    method='euler')
    G_inhib = NeuronGroup(n_inhib, eqs_inh, threshold='v>30', 
                    reset='''v=c; u=u+d_i;''', 
                    method='euler')
    G_excit.v = c
    G_inhib.v = c

    G_excit.oper_ctr[0] = 0

    # synapses: 4 groups - as sharing variables across neuron-groups is not possible in Brian.
    eqs_syn_e_e = '''
    dg1/dt = -g1/tau_g : 1 (clock-driven)
    I_tot_e_e_post = ws_e*g1 : 1 (summed)
    '''
    eqs_syn_e_i = '''
    dg2/dt = -g2/tau_g : 1 (clock-driven)
    I_tot_e_i_post = ws_e*g2 : 1 (summed)
    '''
    eqs_syn_i_e = '''
    dg3/dt = -g3/tau_g : 1 (clock-driven)
    I_tot_i_e_post = -ws_i*g3 : 1 (summed)
    '''
    eqs_syn_i_i = '''
    dg4/dt = -g4/tau_g : 1 (clock-driven)
    I_tot_i_i_post = -ws_i*g4 : 1 (summed)
    '''

    S1 = Synapses(G_excit, G_excit, eqs_syn_e_e, on_pre='g1 = 1', method='euler')
    S2 = Synapses(G_excit, G_inhib, eqs_syn_e_i, on_pre='g2 = 1', method='euler')
    S3 = Synapses(G_inhib, G_excit, eqs_syn_i_e, on_pre='g3 = 1', method='euler')
    S4 = Synapses(G_inhib, G_inhib, eqs_syn_i_i, on_pre='g4 = 1', method='euler')

    # Paper method; hardcode wiring to 10 (random) neurons, for each neuron
    for ind in range(0, n_excit):
        targets = random.sample([i for i in range(n) if i != ind], 10)
        for tar in targets:
            if tar<n_excit:
                S1.connect(i=ind, j=tar)
            else:
                S2.connect(i=ind, j=tar%n_excit)

    for ind in range(0, n_inhib):
        targets = random.sample([i for i in range(n) if i != ind], 10)
        for tar in targets:
            if tar<n_excit:
                S3.connect(i=ind, j=tar)
            else:
                S4.connect(i=ind, j=tar%n_excit)

    @network_operation(dt=1*ms)
    def excite_random_exc_neuron():
        G_excit.I_ext[:] = 0
        idx = int(rand()*(n_excit))  # no n-1 because rand() produces idx=[0, 1)
        G_excit.I_ext[idx] = 100
        
        G_excit.oper_ctr[0] += 1
        
    def assert_network_op_post_run(t):
        assert(int(round(G_excit.oper_ctr[0])) == int(round(t/(1*ms))))

        if(debug):
            print('G_excit.oper_ctr[0]: {}, t/ms: {}'.format(int(round(G_excit.oper_ctr[0])), int(round(t/(ms)))))

    # start recording spikes
    statemon_excit = StateMonitor(G_excit[:], ['I_tot_e', 'I_tot_e_e', 'I_tot_i_e', 'I_ext', 'v', 'u'], record=True, dt=1*ms)
    statemon_inhib = StateMonitor(G_inhib[:], ['I_tot_i', 'I_tot_e_i', 'I_tot_i_i', 'I_ext', 'v', 'u'], record=True, dt=1*ms)
    spikemon_excit = SpikeMonitor(G_excit[:], variables='v')
    spikemon_inhib = SpikeMonitor(G_inhib[:], variables='v')
    t1=tau*steps
    run(t1)
    assert_network_op_post_run(t1)

    stop()
    
    return {0: spikemon_excit, 1: spikemon_inhib, 2: statemon_excit, 3: statemon_inhib}

In [4]:
def network_spike_plot(spikemon_excit, spikemon_inhib):
    plt.figure(figsize=(11, 4))
    plt.subplots_adjust(top=0.99)
    msize=0.2
    n_excit = len(spikemon_excit.spike_trains())
    plt.plot(spikemon_inhib.t/ms, n_excit+spikemon_inhib.i, '.k', markersize=msize)
    plt.plot(spikemon_excit.t/ms, spikemon_excit.i, '.k', markersize=msize)
    plt.xlabel('Time (ms)')
    plt.ylabel('Neuron index');
    plt.tight_layout();
    fname = './figures/' + str(time.time()).replace('.', '_') + '_spikeplot' + '.eps'
    plt.savefig(fname, format='eps', dpi=1200)
#     title('Network spike plot (top 20 % inhibitory, bottom 80 % excitatory neurons)');

In [5]:
def plot_avg_neuron_signals(me, mi):
    fig = figure(figsize=(12,4))
    fig.subplots_adjust(top=0.2) 
    ax1, ax2, ax3, ax4 = fig.subplots(1, 4, sharey=True)
    ax1.grid(True)
    ax2.grid(True)
    # for looking at shorter periods, making the spiking behaviour more visible
    span_index = 200;
    ax1.plot(me.t[:span_index]/ms, me.v[0][:span_index], '-', label='v')
    ax1.plot(me.t[:span_index]/ms, me.u[0][:span_index], '--', label='u')
    ax1.set_xlabel('Time (ms)')
    ax1.set_ylabel('v (mV)')
    ax1.legend(['v', 'u'])
    ax2.plot(me.t[-span_index:]/ms, me.v[0][-span_index:], '-', label='v')
    ax2.plot(me.t[-span_index:]/ms, me.u[0][-span_index:], '--', label='u')
    ax2.set_xlabel('Time (ms)')
    ax3.grid(True)
    ax4.grid(True)
    ax3.plot(mi.t[:span_index]/ms, mi.v[0][:span_index], '-', label='v')
    ax3.plot(mi.t[:span_index]/ms, mi.u[0][:span_index], '--', label='u')
    ax3.set_xlabel('Time (ms)')
    ax4.plot(mi.t[-span_index:]/ms, mi.v[0][-span_index:], '-', label='v')
    ax4.plot(mi.t[-span_index:]/ms, mi.u[0][-span_index:], '--', label='u')
    ax4.set_xlabel('Time (ms)')
#     fig.suptitle('Random excitatory (left pair), and inhibitory (right pair) neuron');
    ax1.title.set_text('Exc. neuron (start)')
    ax2.title.set_text('Exc. neuron (end)')
    ax3.title.set_text('Inh. neuron (start)')
    ax4.title.set_text('Inh. neuron (end)')
    plt.tight_layout();
    fname = './figures/' + str(time.time()).replace('.', '_') + '_neuron_signals' + '.eps'
    plt.savefig(fname, format='eps', dpi=1200)
    
    fig = figure(figsize=(11, 6))
    ax1, ax2 = fig.subplots(1, 2)
    ax1.plot(me.t/ms, np.mean(me.I_tot_e, axis=0))
    ax1.plot(me.t/ms, np.mean(mi.I_tot_i, axis=0))
    ax1.plot(me.t/ms, np.mean(me.I_ext, axis=0))
    ax2.plot(me.t/ms, me.I_tot_e[0])
    ax2.plot(me.t/ms, mi.I_tot_i[0])
    ax2.plot(me.t/ms, me.I_ext[0])
    ax1.set_ylabel('Average neuron currents (mA)')
    ax2.set_ylabel('Random neuron current (mA)')
    ax1.legend([r'$I_{syn_{excit}}$', r'$I_{syn_{inhib}}$'] \
              + ['$I_{ext}$'])
    ax2.legend([r'$I_{syn_{excit}}$', r'$I_{syn_{inhib}}$'] \
              + ['$I_{ext}$'])
    ax1.set_xlabel('Time (ms)')
    ax2.set_xlabel('Time (ms)')
    fname = './figures/' + str(time.time()).replace('.', '_') + '_full_duration_neuron_currents' + '.eps'
#     plt.savefig(fname, format='eps', dpi=1200)
    fig.suptitle('Neuron current signals (full)')
    
    fig = figure(figsize=(12,4))
    ax1, ax2, ax3, ax4 = fig.subplots(1, 4)
    fig.subplots_adjust(top=0.4)
    ax1.plot(me.t[:span_index]/ms, np.mean(me.I_tot_e_e, axis=0)[:span_index])
    ax1.plot(me.t[:span_index]/ms, np.mean(me.I_tot_i_e, axis=0)[:span_index])
    ax1.plot(me.t[:span_index]/ms, np.mean(me.I_ext, axis=0)[:span_index])
    ax2.plot(mi.t[-span_index:]/ms, np.mean(mi.I_tot_e_i, axis=0)[-span_index:])
    ax2.plot(mi.t[-span_index:]/ms, np.mean(mi.I_tot_i_i, axis=0)[-span_index:])
    ax2.plot(mi.t[-span_index:]/ms, np.mean(mi.I_ext, axis=0)[-span_index:])
    ax1.set_ylabel('Neuron currents (mA)')
    ax1.legend([r'$I_{syn_{excit}}$', r'$I_{syn_{inhib}}$'] \
              + ['$I_{ext}$'])
    ax3.plot(me.t[:span_index]/ms, me.I_tot_e_e[0][:span_index])
    ax3.plot(me.t[:span_index]/ms, me.I_tot_i_e[0][:span_index])
    ax3.plot(me.t[:span_index]/ms, me.I_ext[0][:span_index])
    ax4.plot(mi.t[-span_index:]/ms, mi.I_tot_e_i[0][-span_index:])
    ax4.plot(mi.t[-span_index:]/ms, mi.I_tot_i_i[0][-span_index:])
    ax4.plot(mi.t[-span_index:]/ms, mi.I_ext[0][-span_index:])
    ax1.set_xlabel('Time (ms)')
    ax2.set_xlabel('Time (ms)')
    ax3.set_xlabel('Time (ms)')
    ax4.set_xlabel('Time (ms)')
    ax1.grid(True)
    ax2.grid(True)
    ax3.grid(True)
    ax4.grid(True)
#     fig.suptitle('Neuronal currents (left pair: average, right: random neurons)');
    ax1.title.set_text('Pop. avg. (start)')
    ax2.title.set_text('Pop. avg. (end)')
    ax3.title.set_text('Exc. neuron (start)')
    ax4.title.set_text('Inh. neuron (end)')
    plt.tight_layout();
    fname = './figures/' + str(time.time()).replace('.', '_') + '_neuron_currents' + '.eps'
    plt.savefig(fname, format='eps', dpi=1200)

In [6]:
def evaluate(monitors, n, do_plot, debug=False, nperseg=4096):
    spikemon_excit = monitors[0]; spikemon_inhib = monitors[1];
    statemon_excit = monitors[2]; statemon_inhib = monitors[3];

    t = round(statemon_excit.t[-1]/ms)-round(statemon_excit.t[0]/ms)
    avg_e = spikemon_excit.num_spikes/(n*t); avg_i = spikemon_inhib.num_spikes/(n*t)
    
    neuron_signals = np.append(statemon_excit.v, statemon_inhib.v, axis=0)
    freq_max_en = determine_main_frequency(t=statemon_excit.t, x=neuron_signals, do_plot=do_plot, nperseg=nperseg)

    print("frequency with highest energy: {}".format(freq_max_en))
    
    chaotic_behaviour = avg_e > 500. or avg_i > 500.
    if(chaotic_behaviour):
        print('WARNING: Chaotic behaviour occurred..')
    elif(do_plot):
        network_spike_plot(spikemon_excit, spikemon_inhib)
        plot_avg_neuron_signals(statemon_excit, statemon_inhib)
        
    return [freq_max_en, avg_e, avg_i, chaotic_behaviour]

In [7]:
def run_once_and_plot(a, b, ws_e=10, ws_i=10, n=500, tau_g=5.0*ms, steps=512):
    params_str = 'a={}, b={}, ws_e={}, ws_i={}, n={}, tau_g={}'.format(a,b,ws_e,ws_i,n,tau_g)
    print('Parameters:', params_str)
    
    # Run
    monitors = create_and_run_network_model(a, b, ws_e, ws_i, n, tau_g, steps)
    evaluate(monitors, n=n, do_plot=True, nperseg=min(4096, steps))
    return monitors

In [8]:
def perform_network_firing_rate_test(a, b, ws=10, n=500, tau_g=5.0*ms, steps=4096, \
                                     verbose=True, do_plot=True, debug=False):
    f_max_en = 0
    chaotic_behaviour = False
    run_ctr = 0
    plot_title = 'a={}, b={}, ws={}, n={}, tau_g={}'.format(a,b,ws,n,tau_g)
    
    while(not (f_max_en > 0) and not chaotic_behaviour):
        # Run
        monitors = create_and_run_network_model(a=a, b=b, ws_e=ws, ws_i=ws, n=n, tau_g=tau_g, steps=steps)
        
        # Evaluate
        [f_max_en, avg_e, avg_i, chaotic_behaviour] = evaluate(monitors, n, do_plot, plot_title)
        
        run_ctr += 1

    return [f_max_en, avg_e, avg_i, run_ctr]

In [9]:
def avg_network_firing_rate_over_N_runs(a, b, ws=10, n=500, tau_g=5.0*ms, steps=4096,
                                        verbose=False, do_plot_every_iter=False, N=30, debug=False, 
                                        log_fname_postfix='', plot_once_per_exp=True, 
                                        logging_on=True, log_version='30'):
    parameters = [a, b, ws, n, tau_g]
    avg_excits = []
    avg_inhibs = []
    f_max_ens = []
    
    log_str = 'Running experiment over N={} runs for parametrisation: a={}, b={}, ws={}, n={}, tau_g={}'.format(N, a, b, ws, n, tau_g)
    if(verbose):
        print(log_str)
    if(logging_on):
        write_to_logfile(parameters, log_str, log_version, log_fname_postfix)
    
    for ctr in range(0, N):
        do_plot = (ctr==0 and plot_once_per_exp) or do_plot_every_iter
        [f_max_en, cur_avg_e, cur_avg_i, run_ctr] = \
        perform_network_firing_rate_test(a=a, b=b, ws=ws, n=n, tau_g=tau_g, steps=steps, \
                                         verbose=verbose, do_plot=do_plot, debug=debug)
        f_max_ens += [f_max_en]
        avg_excits += [cur_avg_e]
        avg_inhibs += [cur_avg_i]
        
        avg_rate = cur_avg_e * 0.8 + cur_avg_i * 0.2
        log_str = 'run #{}, cur. f_max_en: {}, avg. rate: {}, cur_avg_e_rate: {}, cur_avg_i: {}'.format(ctr, f_max_en, avg_rate, cur_avg_e, cur_avg_i)
        if(logging_on):
            write_to_logfile(parameters, log_str, log_version, log_fname_postfix)
        if(verbose):
            print(log_str)
    
    mean_f_max_en = np.mean(f_max_ens)
    std_f_max_en = np.std(f_max_ens)
    
    mean_e = np.mean(avg_excits)
    std_e = np.std(avg_excits)
    mean_i = np.mean(avg_inhibs)
    std_i = np.std(avg_inhibs)
    
    log_str = 'mean_f_max_en: {}, std_f_max_en: {}, mean_e: {}, std_e: {}, mean_i: {}, std_i: {}'.format(mean_f_max_en, std_f_max_en, mean_e, std_e, mean_i, std_i)
    if(logging_on):
        write_to_logfile(parameters, log_str, log_version, log_fname_postfix)
    print('parameters: {}, log_str: {}'.format(parameters, log_str))
    return [mean_f_max_en, std_f_max_en, mean_e, std_e, mean_i, std_i]

In [10]:
# ================================= ANALYSIS: =========================================

In [11]:
import datetime as dt
from brian2 import figure, grid, axhline, xlabel, ylabel, plot, legend, errorbar, title, scatter

In [12]:
def write_to_logfile(params, log_str, version='unknown_version', opt_fname_postfix=''):
    fname = './results/'+version
    if(opt_fname_postfix!=''):
        fname += '_' + opt_fname_postfix
    fname += '.txt'
    
    prefix = '[{}] (a={}, b={}, ws={}, n={}, tau_g={})'.format(dt.datetime.now(), params[0], params[1], params[2], params[3], params[4])
    full_str = prefix + ' ' + log_str + '\n'
    with open(fname, 'a') as f:
        f.write(full_str)

In [13]:
def unwrap_var_value_from_param_str(var_str, params_str):
    tmp = params_str.split(',')
    tmp = list(filter(lambda s: var_str in s, tmp))[0]
    val_str = tmp.split('=')[1]
    return float(val_str)

def parser_helper_get_values_for(var_str, parameters):
    vals = []
    for params_arr in parameters:
        vals += [unwrap_var_value_from_param_str(var_str, params_arr[1])]
        
    return vals

def get_description_from_str_without(var_str, params_str):
    tmp = params_str.split(',')
    tmp = list(filter(lambda s: var_str not in s, tmp))
    res = ''
    for substr in tmp:
        res += substr + ', '
    return res[:-2]

In [14]:
def read_from_logfile(fname, const_value, const_str='a', read_all=False):
    with open(fname) as file:
        file_contents = file.read()

    tmp = file_contents.split('\n')
    tmp = list(filter(lambda s: 'mean_f_max_en' in s, tmp))
    
    mean_f_max_ens = []
    std_f_max_ens = []
    freq_exc = []
    std_exc = []
    freq_inh = []
    std_inh = []
    parameters = []
    parsed_str_ctr = 0
    for line_str in tmp:
        # parse time and parametrisation
        res_line_str = line_str.split(') ')[1]
        parameters_arr = line_str.split(') ')[0].split('] (')
        datetime_str = parameters_arr[0][1:]
        parametrisation_str = parameters_arr[1]
        
        if(read_all or const_value == unwrap_var_value_from_param_str(const_str, parametrisation_str)):
            parameters += [[datetime_str, parametrisation_str]]

            # parse results after N(=30 by default) converged runs
            tmp_vars = res_line_str.split(', ')
            assert(tmp_vars[0].split(' ')[0].index('mean_f_max_en') > -1)
            assert(tmp_vars[1].split(' ')[0].index('std_f_max_en') > -1)
            assert(tmp_vars[2].split(' ')[0].index('mean_e') > -1)
            assert(tmp_vars[3].split(' ')[0].index('std_e') > -1)
            assert(tmp_vars[4].split(' ')[0].index('mean_i') > -1)
            assert(tmp_vars[5].split(' ')[0].index('std_i') > -1)
            mean_f_max_ens += [float(tmp_vars[0].split(' ')[1])]
            std_f_max_ens += [float(tmp_vars[1].split(' ')[1])]
            freq_exc += [float(tmp_vars[2].split(' ')[1])]
            std_exc += [float(tmp_vars[3].split(' ')[1])]
            freq_inh += [float(tmp_vars[4].split(' ')[1])]
            std_inh += [float(tmp_vars[5].split(' ')[1])]

            parsed_str_ctr += 1
    
    assert(len(mean_f_max_ens) == len(std_f_max_ens))
    assert(len(std_f_max_ens) == len(freq_exc))
    assert(len(freq_exc) == len(std_exc))
    assert(len(std_exc) == len(freq_inh))
    assert(len(freq_inh) == len(std_inh))
    
    return [parameters, mean_f_max_ens, std_f_max_ens, freq_exc, std_exc, freq_inh, std_inh]

In [15]:
def plot_parsed_data(data, var_str='b', const_str='a', const_val=0.02):
    [parameters, pop_rates, std_pop_rates, freq_exc, std_exc, freq_inh, std_inh, avg_rate, avg_std] = data
    vals = parser_helper_get_values_for(var_str, parameters)
    
    figure(figsize=(9, 4))
    ylabel('Average neuronal firing rate ($Hz$)')
    xlabel('$'+var_str+'$-value, ($'+const_str+'='+"{:3.2f}".format(const_val)+'$)')
    grid(True)

    errorbar(vals, freq_exc, yerr=std_exc, fmt='b--*')
    errorbar(vals, freq_inh, yerr=std_inh, fmt='g-o')
    errorbar(vals, avg_rate, yerr=avg_std, fmt='r--s')

    legend(['Excitatory neurons', 'Inhibitory neurons', 'Population average'])
    title('effect of $'+var_str+'$ on average network firing rate,\n' + get_description_from_str_without(var_str, parameters[0][1]));

In [17]:
def generate_combined_tables_file(version_prefix, version_postfix, postfix='_combined', folder_path='./results/'):
    filenames = []
    for i in range(9, 15):
        if(i==10):
            filenames += [folder_path+version_prefix+'table_'+str(i)+version_postfix+'.txt']
        else:
            filenames += [folder_path+version_prefix+'table_'+str(i)+version_postfix+'.txt']
    output_fname = folder_path+version_prefix+'tables'+version_postfix+postfix+'.txt'
    with open(output_fname, 'w') as outfile:
        for fname in filenames:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)
    
    return [output_fname] + filenames

In [19]:
def parse_data_for_tables(version_prefix, version_postfix, postfix='_combined', folder_path='./results/'):
    filenames = generate_combined_tables_file(version_prefix, version_postfix, postfix, folder_path)
    
    all_data = []
    for fname in filenames:
        parsed_data = read_from_logfile(fname, -1, read_all=True);
        all_data += [parsed_data]
    return all_data

In [21]:
def plot_var_product_vs_firing_rate(data, var1_str='a', var2_str='b'):
    [parameters, pop_rates, std_pop_rates, _, _, _, _] = data
    vals1 = parser_helper_get_values_for(var1_str, parameters)
    vals2 = parser_helper_get_values_for(var2_str, parameters)
    
    vars_product = np.multiply(vals1, vals2)
    
    filtered_pop_rates = []
    filtered_vars_product = []
    filtered_std_pop_rates = []
    for i, rate in enumerate(pop_rates):
        if(rate>0 and rate < 60): # currently filter out by threshold for logs with non-synchronous networks saved in the results file
            filtered_pop_rates += [rate]
            filtered_vars_product += [vars_product[i]]
            filtered_std_pop_rates += [std_pop_rates[i]]
    
    filtered_vars_product = np.asarray(filtered_vars_product)
    filtered_pop_rates = np.asarray(filtered_pop_rates)
    filtered_std_pop_rates = np.asarray(filtered_std_pop_rates)
    fig, ax = plt.subplots()
    ax.plot(filtered_vars_product, filtered_pop_rates, 'bd');

    ax.grid(True)
    ax.set_ylabel('Firing rate ($Hz$)')
    ax.set_xlabel('Parameter product ($ab$)');
    return [vars_product, pop_rates, std_pop_rates, \
            filtered_vars_product, filtered_pop_rates, filtered_std_pop_rates]